# Sandbox notebook: 
    
This notebook is for testing functions in peices while also viewing their output, plotting etc., (the things you're not really supposed to have in tests)
It's designed to run independent of any other parts of the package, just in case.

First order of business: Try to get the model creation function to accept multiple inputs.

In [ ]:
# dependencies and data visualization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
import seaborn as sns
import arviz as az

The inputs for our new function are:
   - num_neutrinos: array of floats
        - The number of muon neutrinos shot at the detector in each energy bin
   - num_nue: array of floats
        - The number of electron neutrinos detected in each energy bin
   - energy_bins: array of floats
       - bins edges for the energies we've biined data into
   - est_ss2t: float between 0 and 1
       - estimated ss2t from previous experiments, for use in the prior
   - est_dms: float above 0
       - estimated dms from previous experiments, also for use in prior
       
       
We'll use 
   - num_neutrinos: 600000
   - num_nue: array of floats, calulated below
   - energy_bins: \[0.01, 0.3, 0.7, 1.5, 2\]
   - est_ss2t: 0.5
   - est_dms: 0.8

In [ ]:
# Define all out inputs

UC = 1.27 #Unit conversion factor for the coefficient on dms in the oscillation probability

energy_bins = np.array([0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,1.0,1.2, 1.5, 2]) #(To be used once everything is working properly.)
num_neutrinos = np.full(energy_bins.size-1, 600000) #to fit one rate only, use an int for num_neutrinos

est_ss2t = 0.2
est_dms = 0.8

# Define L and E
# These shouldn't be exact, but for testing, we'll be lazy
E = np.zeros(energy_bins.size-1)
for i in range(E.size):
    E[i] = energy_bins[i]+(energy_bins[i+1]-energy_bins[i])/2
L = 0.5 

num_nue = num_neutrinos*est_ss2t*(np.sin(est_dms*UC*L/E))**2
true_rates =est_ss2t*(np.sin(est_dms*UC*L/E))**2
print(true_rates)

# Begin to define what will eventually be the function that sets up our model

First, we look at taking the things we want to be inputs to the function, and making sure they are in the correct useable form

In [ ]:
# make sure we can get values to use as mean energy from our bin edges when we use a truncated normal
energies = np.zeros(energy_bins.size-1)
sigmas = np.zeros(energy_bins.size-1)
for i in range(energies.size):
    energies[i] = energy_bins[i] + (energy_bins[i+1]-energy_bins[i])/2
    sigmas[i] = (energy_bins[i+1]-energy_bins[i]) # Most energy reconstructions aim for the standard deviation to be about the bin width. Later, we can upadate to make this a keyword argument
    
# When we attempt to use a uniform distribution for energy, these should be the edges      
energies_high = np.array(energy_bins[1:])
energies_low = np.array(energy_bins[:-1])

Now we set up the model

In [ ]:
osc_model = pm.Model()
with osc_model:
        
    # We don't know the exact production point of each neutrino, so we draw from a truncated gaussian (enforcing positive distance travelled)   
    L = pm.TruncatedNormal('L', mu = 0.500, sigma = 0.01, lower = 0.02, upper = 0.6) #units of km
    
    #If this works, we'll have one E distribution for each energy bin
    #E = energies #
    pm.TruncatedNormal('E', mu = energies, sigma = sigmas, lower = energies_low, shape=energies.shape[0]) #units of GeV
        
    # Priors for unknown model parameters, centered on a prior estimate of ss2t, dms
    # Est_ss2t, est_dms defined in previous cell, will be input parameters in our function
    ss2t = pm.TruncatedNormal('sin^2_2theta', mu = est_ss2t, sigma = 0.1, lower = 0, upper = 1 ) #pm.Uniform('sin^2_2theta', 0.0001, 1)
    dms = pm.TruncatedNormal('delta_m^2', mu = est_dms, sigma = 0.1, lower = 0, shape = (1))
        
    # In the large n limit, because the number of oscillations is low, we use a Poisson approximation
    rate = pm.Deterministic('rate', ss2t*(np.sin(dms*(UC*L)/E))**2)
        
    #Likelihood of observations
    measurements = pm.Poisson('nue_flux', mu = rate*num_neutrinos, observed = num_nue)

In [ ]:
osc_model


Okay, this looks good! Next order of business:
    
## Get model comparison to work with the multiple-input model

In [ ]:
num_draws = 1000
tuning_steps = 1000
with osc_model:
    trace = pm.sample(num_draws, tune= tuning_steps)
    az.plot_trace(trace)

We see that we're not properly telling the sampler that there should be multiple values for E, and conseqently, the rate. What we'd like to do is have a value of E and rate for each bin, and the same value across all bins for $sin^2(2\theta)$, $\Delta m^2$, and L. 

Just in case, we wan to still check the marginal distributions of $sin^2(2\theta)$, $\Delta m^2$. However, we've defined some of these quantities incorrectly, so it's not workgin (see below)

In [ ]:
params = pm.trace_to_dataframe(trace)
params_0 = pm.trace_to_dataframe(trace, chains =  0)
params_1 = pm.trace_to_dataframe(trace, chains =  1)
params_2 = pm.trace_to_dataframe(trace, chains =  2)
params_3 = pm.trace_to_dataframe(trace, chains =  2)

In [ ]:
means = [params_0.mean(), params_1.mean(), params_2.mean(), params_3.mean()]
headers = ['params_0', 'params_1', 'params_2', 'params_3']
means_df = pd.concat(means, axis=1, keys=headers)

In [ ]:
means_df.loc[:]

In [ ]:
joint_kde = sns.jointplot(x='delta_m^2__0', y='sin^2_2theta', data=params, kind='scatter', s=0.2)

#joint_kde.ax_marg_x.set_xlim(0, 10)
#joint_kde.ax_marg_y.set_ylim(0, 1)

plt.show()

Now we want to confirm that for a given energy bin, each chain is finding $\Delta m^2$, $sin^2(2\theta)$ values that give rates consistent with those in the other chains. 

In [ ]:
energy_0 = np.full(4, energies[0])
energy_2 = np.full(4, energies[2])
energy_3 = np.full(4, energies[3])
energy_5 = np.full(4, energies[5])
energy_7 = np.full(4, energies[7])
energy_9 = np.full(4, energies[9])

plt.scatter(energy_0, means_df.loc['rate__0'])
plt.scatter(energy_2, means_df.loc['rate__2'])
plt.scatter(energy_3, means_df.loc['rate__3'])
plt.scatter(energy_5, means_df.loc['rate__5'])
plt.scatter(energy_7, means_df.loc['rate__7'])
plt.scatter(energy_9, means_df.loc['rate__9'])

We note that the when the chains have lengths which agree exactly, the rates which come out of the fit agree as well. When there are slight differences in length between the chains, the largest differences between estimated rates appear in the 0.2 to 0.8 GeV region, exactly where we expect to be most sensitive to the low-energy excess.This is to be expected given the L dependence in the $sin^2(1.27\Delta m^2 L/E)$ of the oscillation formula, and illustrates one of the difficulties inherent in short-baseline experiments.

In [ ]:
df_trace = pm.trace_to_dataframe(trace)
q = df_trace.quantile([0.16,0.50,0.84], axis=0)
print("delta_m^2 = {:.2f} + {:.2f} - {:.2f}".format(q['delta_m^2__0'][0.50], 
                                            q['delta_m^2__0'][0.84]-q['delta_m^2__0'][0.50],
                                            q['delta_m^2__0'][0.50]-q['delta_m^2__0'][0.16]))
print("sin^2_2theta = {:.1f} + {:.1f} - {:.1f}".format(q['sin^2_2theta'][0.50], 
                                            q['sin^2_2theta'][0.84]-q['sin^2_2theta'][0.50],
                                            q['sin^2_2theta'][0.50]-q['sin^2_2theta'][0.16]))
    
    

In [ ]:
E = 1.0
L = 0.54
ss2t = np.linspace(0, 1, 1000)
rate = [0.1, 0.2, 0.5, 0.7]
dms_0 = E/(UC*L) * np.arcsin(np.sqrt(rate[0]/ss2t))
dms_3 = E/(UC*L) * np.arcsin(np.sqrt(rate[3]/ss2t))


plt.plot(ss2t, dms_0, color='red')
plt.plot(ss2t, dms_3, color='blue')

In [ ]:
means_df

In [ ]:
L = means_df.loc['L']
ss2t = np.linspace(0.0, 1, 1000)

def give_dms(energy, L, rate, ss2t):
    dms = energy/(UC*L) * np.arcsin(np.sqrt(rate/ss2t))
    return dms

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot()
ax1.set_ylabel('delta_m^2')
ax1.set_xlabel('sin^2_2theta')

for i in [0]:#play around with this! Choose any energy bin to investigate (I recommend only one or two at a time, otherwise the legend takes over.)
    E_str = "{:.2f}".format(energies[i])
    labe = 'mean at E = ' + E_str
    ax1.plot(ss2t, give_dms(energies[i], params.mean().loc['L'], params.mean().loc[rate_str], ss2t), label = labe)
    ax1.scatter(params.mean().loc['sin^2_2theta'], params.mean().loc['delta_m^2__0'])      
    for chain in range(0, 4):
        rate_str = 'rate__' + str(i)
        rate = means_df.loc[rate_str]
        L_str = "{:.3f}".format(L[chain])
        label_str = 'L = ' + L_str +', E = '+ E_str + ', chain ' + str(chain)
        ax1.plot(ss2t, give_dms(energies[i], L[chain], rate[chain], ss2t), label = label_str)
        ax1.scatter(means_df.loc['sin^2_2theta'][chain], means_df.loc['delta_m^2__0'][chain])
ax1.legend()